# 示例

## 按指定通道、经纬度范围、分辨率读取数据

In [1]:
from fy4a import AGRI_L1

In [2]:
file_path = r'..\..\fy4\FY4A-_AGRI--_N_REGC_1047E_L1-_FDI-_MULT_NOM_20200911164500_20200911164917_4000M_V0001.HDF'
geo_desc = [17, 54, 73, 135, 0.02]  # 顺序为南、北、西、东、分辨率，即[lat_s, lat_n, lon_w, lon_e, resolution]

In [3]:
file = AGRI_L1(file_path, geo_desc)
Channel01_reflectance = file.extract('Channel01')
Channel01_reflectance

<xarray.DataArray 'Channel01_reflectance' (lat: 1851, lon: 3101)>
array([[    nan,     nan,     nan, ...,     nan,     nan,     nan],
       [    nan,     nan,     nan, ...,     nan,     nan,     nan],
       [    nan,     nan,     nan, ...,     nan,     nan,     nan],
       ...,
       [0.00575, 0.00575, 0.0065 , ..., 0.006  , 0.0075 , 0.0075 ],
       [0.0095 , 0.0095 , 0.011  , ..., 0.00625, 0.00675, 0.00675],
       [0.0095 , 0.0095 , 0.0095 , ..., 0.00625, 0.00675, 0.00675]])
Coordinates:
  * lat      (lat) float64 54.0 53.98 53.96 53.94 ... 17.06 17.04 17.02 17.0
  * lon      (lon) float64 73.0 73.02 73.04 73.06 ... 134.9 135.0 135.0 135.0
Attributes:
    units:    100%

上例中出现`nan`是数据问题，据说是因为秋分日防止仪器被太阳直射所以观测范围变小了

**注意**

`geo_desc`可以在传给构造函数`AGRI_L1()`，也可以传给实例方法`.extract()`，即上例等价于

```python
file = AGRI_L1(file_path)
Channel01_reflectance = file.extract('Channel01', geo_desc=geo_desc)
Channel01_reflectance
```

可以在每次调用实例方法`.extract()`时传入不同合法的`geo_desc`，每次传入新的`geo_desc`，将覆盖旧的`geo_desc`，如下所示

In [4]:
file = AGRI_L1(file_path, [17, 54, 73, 135, 0.02])
print('沿用实例化时传入的geo_desc=[17, 54, 73, 135, 0.02]\n')
print(file.extract('Channel01'))
print('\n\n采用新的geo_desc=[10, 20, 80, 90, 0.1]\n')
print(file.extract('Channel02', geo_desc=[10, 20, 80, 90, 0.1]))
print('\n\n沿用更新后的geo_desc\n')
print(file.extract('Channel03'))

沿用实例化时传入的geo_desc=[17, 54, 73, 135, 0.02]

<xarray.DataArray 'Channel01_reflectance' (lat: 1851, lon: 3101)>
array([[    nan,     nan,     nan, ...,     nan,     nan,     nan],
       [    nan,     nan,     nan, ...,     nan,     nan,     nan],
       [    nan,     nan,     nan, ...,     nan,     nan,     nan],
       ...,
       [0.00575, 0.00575, 0.0065 , ..., 0.006  , 0.0075 , 0.0075 ],
       [0.0095 , 0.0095 , 0.011  , ..., 0.00625, 0.00675, 0.00675],
       [0.0095 , 0.0095 , 0.0095 , ..., 0.00625, 0.00675, 0.00675]])
Coordinates:
  * lat      (lat) float64 54.0 53.98 53.96 53.94 ... 17.06 17.04 17.02 17.0
  * lon      (lon) float64 73.0 73.02 73.04 73.06 ... 134.9 135.0 135.0 135.0
Attributes:
    units:    100%


采用新的geo_desc=[10, 20, 80, 90, 0.1]

<xarray.DataArray 'Channel02_reflectance' (lat: 101, lon: 101)>
array([[4.07453626e-10, 4.07453626e-10, 4.07453626e-10, ...,
        4.07453626e-10, 4.07453626e-10, 4.07453626e-10],
       [4.07453626e-10, 4.07453626e-10, 4.07453626e

## 指定插值方式

在实例方法`.extract()`中传入`interp_method`参数用于指定投影转换时的插值方法，只支持`nearest`和`linear`，默认`nearest`。如4000M数据插值到0.04°可以默认用`nearest`，若要插值到0.001°可能用`linear`更合适。

In [5]:
file.extract('Channel01', interp_method='linear')

<xarray.DataArray 'Channel01_reflectance' (lat: 101, lon: 101)>
array([[0.01079856, 0.0081892 , 0.00904951, ..., 0.00858425, 0.00771904,
        0.008345  ],
       [0.00790092, 0.00795911, 0.0068798 , ..., 0.00715458, 0.00764044,
        0.00716909],
       [0.00661715, 0.0061083 , 0.00800229, ..., 0.00850487, 0.00884231,
        0.00875865],
       ...,
       [0.00834087, 0.00793969, 0.00889885, ..., 0.00644774, 0.00768745,
        0.00822994],
       [0.00891195, 0.00955382, 0.00799107, ..., 0.00790397, 0.00917957,
        0.00848902],
       [0.00689803, 0.00653203, 0.00830225, ..., 0.00804647, 0.0077232 ,
        0.00660674]])
Coordinates:
  * lat      (lat) float64 20.0 19.9 19.8 19.7 19.6 ... 10.4 10.3 10.2 10.1 10.0
  * lon      (lon) float64 80.0 80.1 80.2 80.3 80.4 ... 89.6 89.7 89.8 89.9 90.0
Attributes:
    units:    100%

## 指定定标方式

在实例方法`.extract()`中传入`calibration`参数用于指定定标方式，支持`dn`、`reflectance`、`radiance`和`brightness_temperature`，默认`reflectance`，`dn`表示不定标。


第1\~6通道仅支持`dn`、`reflectance`两种定标方式；第7\~14通道仅支持`dn`、`radiance`和`brightness_temperature`三种定标方式。

第1\~6通道，用定标系数计算反射率`reflectance`，第7\~14通道用定标系数计算辐亮度`radiance`，用定标表插值计算亮温`brightness_temperature`。

In [6]:
file.extract('Channel12', calibration='brightness_temperature')

<xarray.DataArray 'Channel12_brightness_temperature' (lat: 101, lon: 101)>
array([[209.30000305, 210.5       , 211.3999939 , ..., 288.        ,
        286.1000061 , 285.6000061 ],
       [217.5       , 216.19999695, 212.19999695, ..., 289.1000061 ,
        287.        , 287.1000061 ],
       [223.80000305, 224.5       , 221.69999695, ..., 290.1000061 ,
        289.5       , 289.70001221],
       ...,
       [267.6000061 , 260.79998779, 254.80000305, ..., 244.30000305,
        240.19999695, 236.80000305],
       [255.1000061 , 253.        , 251.6000061 , ..., 233.69999695,
        226.80000305, 243.69999695],
       [252.6000061 , 251.        , 251.80000305, ..., 214.1000061 ,
        218.1000061 , 245.1000061 ]])
Coordinates:
  * lat      (lat) float64 20.0 19.9 19.8 19.7 19.6 ... 10.4 10.3 10.2 10.1 10.0
  * lon      (lon) float64 80.0 80.1 80.2 80.3 80.4 ... 89.6 89.7 89.8 89.9 90.0
Attributes:
    units:    K

In [7]:
file.extract('Channel12', calibration='radiance')

<xarray.DataArray 'Channel12_radiance' (lat: 101, lon: 101)>
array([[1.40532881, 1.45719881, 1.49310881, ..., 8.000799  , 7.75341899,
        7.68558899],
       [1.78437882, 1.71654882, 1.52901882, ..., 8.132469  , 7.869129  ,
        7.877109  ],
       [2.11554883, 2.15943883, 1.99983883, ..., 8.27211901, 8.19231901,
        8.22024901],
       ...,
       [5.61477893, 4.92849891, 4.3659089 , ..., 3.48810887, 3.18087886,
        2.93349886],
       [4.3978289 , 4.21029889, 4.08660889, ..., 2.72601885, 2.29110884,
        3.44421887],
       [4.17438889, 4.03074889, 4.10256889, ..., 1.61679882, 1.81230882,
        3.54795887]])
Coordinates:
  * lat      (lat) float64 20.0 19.9 19.8 19.7 19.6 ... 10.4 10.3 10.2 10.1 10.0
  * lon      (lon) float64 80.0 80.1 80.2 80.3 80.4 ... 89.6 89.7 89.8 89.9 90.0
Attributes:
    units:    mW/ (m2 cm-1 sr)

In [8]:
file.extract('Channel12', calibration='dn')

<xarray.DataArray 'NOMChannel12' (lat: 101, lon: 101)>
array([[3733., 3720., 3711., ..., 2080., 2142., 2159.],
       [3638., 3655., 3702., ..., 2047., 2113., 2111.],
       [3555., 3544., 3584., ..., 2012., 2032., 2025.],
       ...,
       [2678., 2850., 2991., ..., 3211., 3288., 3350.],
       [2983., 3030., 3061., ..., 3402., 3511., 3222.],
       [3039., 3075., 3057., ..., 3680., 3631., 3196.]])
Coordinates:
  * lat      (lat) float64 20.0 19.9 19.8 19.7 19.6 ... 10.4 10.3 10.2 10.1 10.0
  * lon      (lon) float64 80.0 80.1 80.2 80.3 80.4 ... 89.6 89.7 89.8 89.9 90.0
Attributes:
    units:    DN

In [9]:
file.extract('Channel12')

ValueError: Channel12没有reflectance的定标方式